https://www.youtube.com/watch?v=XzAryDJTi1M

https://www.youtube.com/watch?v=5HuOGZEu2HY

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

In [ ]:
adata = sc.read_10x_h5('lung1.h5')
sc.pp.filter_cells(adata,min_genes=200)
sc.pp.filter_genes(adata,min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, 
                           log1p=False, inplace=True)
upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, .98)
lower_lim = np.quantile(adata.obs.n_genes_by_counts.values, .02)
adata = adata[(adata.obs.n_genes_by_counts.values < upper_lim) &
              (adata.obs.n_genes_by_counts.values > lower_lim)]
adata = adata[adata.obs.pct_counts_mt < 20]
sc.pp.normilize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=20)
sc.tl.leiden(adata, resolution = 0.25)
